In [1]:
%cd ..
%pwd

/tmp/working


'/tmp/working'

In [2]:
# retrieval
class Config:
    sim_model = "BAAI/bge-small-en"
    sim_max_length = 384

    batch_size = 32
    num_sentences_include = 20
    doc_top_k = 3
    window_size = 5
    sliding_size = 4

    # index
    wiki_index_path = "preprocessed/320_doc_index/001/all.parquet"
    index_path = "preprocessed/320_doc_index/001/ivfpq_index.faiss"

    # wiki b
    wiki_dir = "input/llm-science-wikipedia-data-b"


cfg = Config()

In [3]:
import ctypes
import gc
import os
import re
import sys
from collections.abc import Iterable

import blingfire as bf
import faiss
import numpy as np
import pandas as pd
import torch
from faiss import read_index, write_index
from sentence_transformers import SentenceTransformer
from tqdm.auto import tqdm

libc = ctypes.CDLL("libc.so.6")

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [4]:
def extract_chunk_by_sliding_window(text_list: list[str], window_size: int, sliding_size: int) -> list[str]:
    """
    text のリストをsliding windowで結合する。window_size個のtextが含まれるまで結合し、sliding_size個ずつずらして結合する。
    """
    chunks = []
    for i in range(0, len(text_list), sliding_size):
        chunk = " ".join(text_list[i : i + window_size])
        chunks.append(chunk)
    return chunks


def extract_sections(text: str) -> list[tuple[str, str]]:
    pattern = re.compile(r"#{2,}\s?(.*?)\s?#{2,}")
    sections = []

    matches = list(pattern.finditer(text))
    start_idx = 0

    if len(matches) == 0:
        sections.append(("", text))
        return sections

    for i, match in enumerate(matches):
        if i == 0:
            end_idx = match.start()
            sections.append(("", text[start_idx:end_idx].strip()))

        start_idx = match.end()
        end_idx = matches[i + 1].start() if i + 1 < len(matches) else len(text)
        section = (match.group(1).strip(), text[start_idx:end_idx].strip())
        if section[0] not in ["See also", "References", "Further reading", "External links"]:
            sections.append(section)

        start_idx = end_idx

    # 空のtextの場合は飛ばす
    sections = [section for section in sections if len(section[1].split(" ")) >= 3]
    return sections


def sentencize(
    titles: Iterable[str],
    documents: Iterable[str],
    document_ids: Iterable,
    window_size: int = 3,
    sliding_size: int = 2,
    filter_len: int = 5,
    filter_len_max: int = 500,
    disable_progress_bar: bool = False,
) -> pd.DataFrame:
    """
    Split a document into sentences. Can be used with `sectionize_documents`
    to further split documents into more manageable pieces. Takes in offsets
    to ensure that after splitting, the sentences can be matched to the
    location in the original documents.

    :param documents: Iterable containing documents which are strings
    :param document_ids: Iterable containing document unique identifiers
    :param filter_len: Minimum character length of a sentence (otherwise filter out)
    :return: Pandas DataFrame containing the columns `document_id`, `text`, `section`, `offset`
    """

    document_sentences = []
    for title, document, document_id in tqdm(
        zip(titles, documents, document_ids), total=len(documents), disable=disable_progress_bar
    ):
        try:
            # chunk にまとめる
            ## 念のため改行をスペースに変換
            document = document.replace("\n", " ")
            _, sentence_offsets = bf.text_to_sentences_and_offsets(document)
            section_sentences = []
            for o in sentence_offsets:
                if filter_len < o[1] - o[0] and o[1] - o[0] < filter_len_max:
                    section_sentences.append(document[o[0] : o[1]])
            chunks = extract_chunk_by_sliding_window(section_sentences, window_size, sliding_size)

            for chunk in chunks:
                row = {}
                row["document_id"] = document_id
                row["text"] = f"{title} > {chunk}"
                row["offset"] = (0, 0)
                document_sentences.append(row)
        except:
            continue
    return pd.DataFrame(document_sentences)


def sectionize_documents(
    titles: Iterable[str],
    documents: Iterable[str],
    document_ids: Iterable,
    disable_progress_bar: bool = False,
) -> pd.DataFrame:
    """
    Obtains the sections of the imaging reports and returns only the
    selected sections (defaults to FINDINGS, IMPRESSION, and ADDENDUM).

    :param documents: Iterable containing documents which are strings
    :param document_ids: Iterable containing document unique identifiers
    :param disable_progress_bar: Flag to disable tqdm progress bar
    :return: Pandas DataFrame containing the columns `document_id`, `text`, `offset`
    """
    processed_documents = []
    for title, document_id, document in tqdm(
        zip(titles, document_ids, documents), total=len(documents), disable=disable_progress_bar
    ):
        row = {}
        text, start, end = (document, 0, len(document))
        row["document_id"] = document_id
        row["text"] = text
        row["offset"] = (start, end)

        processed_documents.append(row)

    _df = pd.DataFrame(processed_documents)
    if _df.shape[0] > 0:
        return _df.sort_values(["document_id", "offset"]).reset_index(drop=True)
    else:
        return _df


def relevant_title_retrieval(
    df: pd.DataFrame,
    index_path: str,
    model: SentenceTransformer,
    top_k: int = 3,
    batch_size: int = 32,
) -> pd.DataFrame:
    sentence_index = read_index(index_path)  # index 読み込み
    # res = faiss.StandardGpuResources()  # use a single GPU
    # co = faiss.GpuClonerOptions()
    # co.useFloat16 = True
    # sentence_index = faiss.index_cpu_to_gpu(res, 0, sentence_index, co)
    sentence_index.nprobe = 10
    prompt_embeddings = model.encode(
        df.prompt_answer_stem.values,
        batch_size=batch_size,
        device="cuda",
        show_progress_bar=True,
        # convert_to_tensor=True,
        normalize_embeddings=True,
    )
    prompt_embeddings = prompt_embeddings.astype(np.float32)
    # prompt_embeddings = prompt_embeddings.detach().cpu().numpy()
    search_score, search_index = sentence_index.search(prompt_embeddings, top_k)
    # res.noTempMemory()
    # del res
    del sentence_index
    del prompt_embeddings
    _ = gc.collect()
    libc.malloc_trim(0)
    return search_score, search_index


def get_wikipedia_file_data(
    search_score: np.ndarray,
    search_index: np.ndarray,
    wiki_index_path: str,
) -> pd.DataFrame:
    wiki_index_df = pd.read_parquet(wiki_index_path, columns=["id", "file"])
    wikipedia_file_data = []
    for i, (scr, idx) in tqdm(enumerate(zip(search_score, search_index)), total=len(search_score)):
        _df = wiki_index_df.loc[idx].copy()
        _df["prompt_id"] = i
        wikipedia_file_data.append(_df)
    wikipedia_file_data = pd.concat(wikipedia_file_data).reset_index(drop=True)
    wikipedia_file_data = (
        wikipedia_file_data[["id", "prompt_id", "file"]]
        .drop_duplicates()
        .sort_values(["file", "id"])
        .reset_index(drop=True)
    )
    ## Save memory - delete df since it is no longer necessary
    del wiki_index_df
    _ = gc.collect()
    libc.malloc_trim(0)
    return wikipedia_file_data


def get_full_text_data(
    wikipedia_file_data: pd.DataFrame,
    wiki_dir: str,
):
    ## Get the full text data
    wiki_text_data = []
    for file in tqdm(wikipedia_file_data.file.unique(), total=len(wikipedia_file_data.file.unique())):
        _id = [str(i) for i in wikipedia_file_data[wikipedia_file_data["file"] == file]["id"].tolist()]
        _df = pd.read_parquet(f"{wiki_dir}/{file}", columns=["id", "title", "text"])
        _df_temp = _df[_df["id"].isin(_id)].copy()
        del _df
        _ = gc.collect()
        libc.malloc_trim(0)
        wiki_text_data.append(_df_temp)
    wiki_text_data = pd.concat(wiki_text_data).drop_duplicates().reset_index(drop=True)
    _ = gc.collect()
    libc.malloc_trim(0)
    return wiki_text_data


def extract_contexts_from_matching_pairs(
    df: pd.DataFrame,
    processed_wiki_text_data: pd.DataFrame,
    wikipedia_file_data: pd.DataFrame,
    wiki_data_embeddings: np.ndarray,
    question_embeddings: np.ndarray,
    num_sentences_include: int = 5,
):
    results = {"contexts": [], "sim_min": [], "sim_max": [], "sim_mean": [], "sim_std": [], "sim_num": []}
    for r in tqdm(df.itertuples(), total=len(df)):
        prompt_id = r.Index
        prompt_indices = processed_wiki_text_data[
            processed_wiki_text_data["document_id"].isin(
                wikipedia_file_data[wikipedia_file_data["prompt_id"] == prompt_id]["id"].values
            )
        ].index.values
        assert prompt_indices.shape[0] > 0
        prompt_index = faiss.index_factory(wiki_data_embeddings.shape[1], "Flat")
        prompt_index.add(wiki_data_embeddings[prompt_indices])
        ## Get the top matches
        ss, ii = prompt_index.search(question_embeddings[np.newaxis, prompt_id], num_sentences_include)
        context = ""
        total_len = 0
        num = 0
        for _s, _i in zip(ss[0], ii[0]):
            if total_len > 1000 or _s >= 1.0:
                break
            text = processed_wiki_text_data.loc[prompt_indices]["text"].iloc[_i]
            context += text + " "
            total_len += len(text.split(" "))
            num += 1
        results["contexts"].append(context)
        results["sim_max"].append(ss[0][:num].max())
        results["sim_min"].append(ss[0][:num].min())
        results["sim_mean"].append(ss[0][:num].mean())
        results["sim_std"].append(ss[0][:num].std())
        results["sim_num"].append(num)

    return results

In [5]:
model = SentenceTransformer(cfg.sim_model, device="cuda")
model.max_seq_length = cfg.sim_max_length
model = model.half()

df = pd.read_csv("input/kaggle-llm-science-exam/test.csv").drop("id", 1).head(10)
df[["A", "B", "C", "D", "E"]] = df[["A", "B", "C", "D", "E"]].fillna("")
df.reset_index(inplace=True, drop=True)

df["answer_all"] = df.apply(lambda x: " ".join([x["A"], x["B"], x["C"], x["D"], x["E"]]), axis=1)
df["prompt_answer_stem"] = df["prompt"] + " " + df["answer_all"]
df["prompt_answer_stem"] = df["prompt_answer_stem"].str.replace('"', "")
df["prompt_answer_stem"] = df["prompt_answer_stem"].str.replace("“", "")
df["prompt_answer_stem"] = df["prompt_answer_stem"].str.replace("”", "")


# title 検索
print("【title 検索】")
search_score, search_index = relevant_title_retrieval(
    df,
    cfg.index_path,
    model,
    top_k=cfg.doc_top_k,
    batch_size=cfg.batch_size,
)

/tmp/ipykernel_90/2703185024.py:5: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  df = pd.read_csv("input/kaggle-llm-science-exam/test.csv").drop("id", 1).head(10)


【title 検索】


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [6]:
search_score

array([[0.10069827, 0.12945409, 0.12975068],
       [0.13511868, 0.15179165, 0.18356284],
       [0.129907  , 0.13123852, 0.14057511],
       [0.15040585, 0.16520883, 0.17001614],
       [0.14506043, 0.14928862, 0.16130355],
       [0.11162052, 0.12615484, 0.14368361],
       [0.18993285, 0.19940323, 0.2052996 ],
       [0.18377444, 0.18975887, 0.19107112],
       [0.17239162, 0.18588975, 0.19105905],
       [0.1497725 , 0.1599384 , 0.16583973]], dtype=float32)

In [7]:
# wikipedia file data 取得 ("id", "prompt_id", "file")
print("【wikipedia file data 取得】")
wikipedia_file_data = get_wikipedia_file_data(
    search_score,
    search_index,
    cfg.wiki_index_path,
)
print(wikipedia_file_data.head())

# wikipedia text data 取得 ("id", "text")
print("【wikipedia text data 取得】")
wiki_text_data = get_full_text_data(
    wikipedia_file_data,
    cfg.wiki_dir,
)
print(wiki_text_data.head())

## Parse documents into sentences
print("【sentencize】")
processed_wiki_text_data = sentencize(
    wiki_text_data.title.values,
    wiki_text_data.text.values,
    wiki_text_data.id.values,
    cfg.window_size,
    cfg.sliding_size,
)


## Get embeddings of the wiki text data
print("【Get embeddings of the wiki text data】")
wiki_data_embeddings = model.encode(
    processed_wiki_text_data.text,
    batch_size=cfg.batch_size,
    device="cuda",
    show_progress_bar=True,
    # convert_to_tensor=True,
    normalize_embeddings=True,
)  # .half()
# wiki_data_embeddings = wiki_data_embeddings.detach().cpu().numpy()
wiki_data_embeddings = wiki_data_embeddings.astype(np.float32)
# print data size(GB) of wiki_data_embeddings

_ = gc.collect()
torch.cuda.empty_cache()

## Combine all answers
print("【Combine all answers】")
question_embeddings = model.encode(
    df.prompt_answer_stem.values,
    batch_size=cfg.batch_size,
    device="cuda",
    show_progress_bar=True,
    # convert_to_tensor=True,
    normalize_embeddings=True,
)
question_embeddings = question_embeddings.astype(np.float32)
df.drop(["answer_all", "prompt_answer_stem"], axis=1, inplace=True)
# question_embeddings = question_embeddings.detach().cpu().numpy()
torch.cuda.empty_cache()

【wikipedia file data 取得】


  0%|          | 0/2 [00:00<?, ?it/s]

         id  prompt_id       file
0  52303418          1  d.parquet
1  21591425          0  m.parquet
2  25675405          1  s.parquet
【wikipedia text data 取得】


  0%|          | 0/3 [00:00<?, ?it/s]

         id                                     title  \
0  52303418                           Dynamic scaling   
1  21591425               Modified Newtonian dynamics   
2  25675405  Self-Similarity of Network Data Analysis   

                                                text  
0  Dynamic scaling (sometimes known as Family-Vic...  
1  Modified Newtonian dynamics (MOND) is a hypoth...  
2  In computer networks, self-similarity is a fea...  
【sentencize】


  0%|          | 0/3 [00:00<?, ?it/s]

【Get embeddings of the wiki text data】


Batches:   0%|          | 0/6 [00:00<?, ?it/s]

【Combine all answers】


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [8]:
wiki_data_embeddings[:2, :10]

array([[-4.1931152e-02,  1.8280029e-02,  1.2161255e-02, -5.9852600e-03,
        -1.3183594e-02,  3.3843994e-02,  1.6510010e-02, -1.4901161e-05,
         7.9650879e-03, -2.2796631e-02],
       [-6.0119629e-02, -1.1695862e-02, -6.3514709e-03, -1.3298035e-02,
        -1.5274048e-02,  3.0944824e-02,  9.7198486e-03,  4.4174194e-03,
         8.0261230e-03, -2.3040771e-02]], dtype=float32)

In [9]:
question_embeddings[:2, :10]

array([[-0.05453491, -0.03013611,  0.00688171, -0.02697754,  0.01304626,
        -0.0038166 ,  0.00697327,  0.02407837, -0.00566483,  0.02145386],
       [-0.04086304, -0.01330566, -0.02146912, -0.01564026,  0.01847839,
        -0.00036621,  0.02664185, -0.01551819,  0.03930664, -0.01786804]],
      dtype=float32)

In [10]:
## Extract contexts from matching pairs
results = extract_contexts_from_matching_pairs(
    df,
    processed_wiki_text_data,
    wikipedia_file_data,
    wiki_data_embeddings,
    question_embeddings,
    num_sentences_include=cfg.num_sentences_include,
)
df["context"] = results["contexts"]
df["sim_max"] = results["sim_max"]
df["sim_min"] = results["sim_min"]
df["sim_mean"] = results["sim_mean"]
df["sim_std"] = results["sim_std"]
df["sim_num"] = results["sim_num"]

df.to_csv("./test_context.csv", index=False)

  0%|          | 0/2 [00:00<?, ?it/s]

In [11]:
# infer
class Config:
    # infer
    max_length = 150
    max_length_valid = 300
    model_path = "dataset/llm-science-models/300_000"
    sep_token = "['SEP']"


cfg = Config()


import gc
import os
import re
from dataclasses import dataclass
from typing import Optional, Union

import numpy as np
import pandas as pd
import torch
from datasets import Dataset
from tqdm.auto import tqdm
from transformers import AutoModelForMultipleChoice, AutoTokenizer, Trainer, TrainingArguments
from transformers.tokenization_utils_base import PaddingStrategy, PreTrainedTokenizerBase


@dataclass
class DataCollatorForMultipleChoice:
    tokenizer: PreTrainedTokenizerBase
    padding: Union[bool, str, PaddingStrategy] = True
    max_length: Optional[int] = None
    pad_to_multiple_of: Optional[int] = None

    def __call__(self, features):
        label_name = "label" if "label" in features[0].keys() else "labels"
        labels = [feature.pop(label_name) for feature in features]
        batch_size = len(features)
        num_choices = len(features[0]["input_ids"])
        flattened_features = [
            [{k: v[i] for k, v in feature.items()} for i in range(num_choices)] for feature in features
        ]
        flattened_features = sum(flattened_features, [])

        batch = self.tokenizer.pad(
            flattened_features,
            padding=self.padding,
            max_length=self.max_length,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors="pt",
        )
        batch = {k: v.view(batch_size, num_choices, -1) for k, v in batch.items()}
        batch["labels"] = torch.tensor(labels, dtype=torch.int64)
        return batch


def clean_text(text):
    text = text.replace('"', "")
    text = text.replace("“", "")
    text = text.replace("”", "")
    return text


test_df = pd.read_csv("test_context.csv")


def preprocess_df(df, mode="train"):
    max_length = cfg.max_length if mode == "train" else cfg.max_length_valid  # 推論時はtokenを長く取る
    df["prompt_with_context"] = (
        df["context"].apply(lambda x: " ".join(x.split()[:max_length])) + f"... {cfg.sep_token} " + df["prompt"]
    )
    df["prompt_with_context"] = df["prompt_with_context"].apply(clean_text)

    # 空を埋める
    options = ["A", "B", "C", "D", "E"]
    for option in options:
        df[option] = df[option].fillna("")
    return df


test_df = preprocess_df(test_df, mode="valid")
test_df["answer"] = "A"
test_ds = Dataset.from_pandas(test_df)


# We'll create a dictionary to convert option names (A, B, C, D, E) into indices and back again

tokenizer = AutoTokenizer.from_pretrained(cfg.model_path)
option_to_index = {option: idx for idx, option in enumerate("ABCDE")}


def preprocess(example):
    first_sentence = [example["prompt_with_context"]] * 5
    second_sentences = [example[option] for option in "ABCDE"]
    tokenized_example = tokenizer(first_sentence, second_sentences, truncation=False)
    tokenized_example["label"] = option_to_index[example["answer"]]

    return tokenized_example


tokenized_test_ds = test_ds.map(
    preprocess,
    batched=False,
    remove_columns=["prompt_with_context", "prompt", "A", "B", "C", "D", "E", "answer"],
)

model = AutoModelForMultipleChoice.from_pretrained(cfg.model_path)
model.eval()


args = TrainingArguments(output_dir="output/tmp", per_device_eval_batch_size=1)
trainer = Trainer(
    model=model,
    args=args,
    tokenizer=tokenizer,
    data_collator=DataCollatorForMultipleChoice(
        tokenizer=tokenizer,
    ),
)


test_predictions = trainer.predict(tokenized_test_ds).predictions

Map:   0%|          | 0/2 [00:00<?, ? examples/s]

You're using a DebertaV2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


In [12]:
test_predictions

array([[-0.4838045 , -1.6397529 , -1.1198299 ,  9.401418  ,  1.4131405 ],
       [ 2.8451388 ,  0.62608135, -1.232299  , -0.84630734, -0.59199804]],
      dtype=float32)